# Обработка данных сетевого трафика
Задача: преобразовать данные о сетевом трафике к удобному виду, допустимому для использования в будущих моделях и алгоритмах.

# Источники
[данные](https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/%D1%83%D0%BF%D1%80%D0%B0%D0%B6%D0%BD%D0%B5%D0%BD%D0%B8%D1%8F/data/synscan.csv) взяты из Интернета

# Изменения
01.12.2020 преобразование данных и оформление блокнота

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/%D1%83%D0%BF%D1%80%D0%B0%D0%B6%D0%BD%D0%B5%D0%BD%D0%B8%D1%8F/data/synscan.csv")
df["tcp_retransmission"] = df.Info.str.find("[TCP Retransmission]")  
df["tcp_retransmission"] = np.where(df["tcp_retransmission"] == 0, True, False)
df.columns = [x.lower() for x in df.columns]
df["info"] = df["info"].str.replace(r"[TCP Retransmission]", "", regex=False)
df["info"] = df["info"].str.strip()
df.head()

,no.,time,source,destination,protocol,length,info,tcp_retransmission
0,1,0.000000,172.16.0.8,64.13.134.52,TCP,58,36050 > 443 [SYN] Seq=0 Win=3072 Len=0 MSS=1460,False
1,2,0.001539,172.16.0.8,64.13.134.52,TCP,58,36050 > 143 [SYN] Seq=0 Win=3072 Len=0 MSS=1460,False
2,3,0.001597,172.16.0.8,64.13.134.52,TCP,58,36050 > 3306 [SYN] Seq=0 Win=2048 Len=0 MSS=...,False
3,4,0.001650,172.16.0.8,64.13.134.52,TCP,58,36050 > 199 [SYN] Seq=0 Win=3072 Len=0 MSS=1460,False
4,5,0.001703,172.16.0.8,64.13.134.52,TCP,58,36050 > 111 [SYN] Seq=0 Win=1024 Len=0 MSS=1460,False


In [4]:
df["SYN"]=df["info"].str.find("SYN")
df["SYN"]=np.where(df["SYN"]>0,True,False)
df["ACK"]=df["info"].str.find("ACK")
df["ACK"]=np.where(df["ACK"]>0,True,False)


In [5]:
df["Seq"]=[df["info"].str[y:z][x-1] for (x,y,z) in zip(df["no."],df["info"].str.find("Seq=")+4,df["info"].str.find("Win"))] #создание столбца со значениями Seq

In [6]:
df["Seq"]=df["Seq"].str.replace(r"Ack=1","",regex=False)
df["Win"]=[df["info"].str[y:y+4][x-1] for (x,y) in zip(df["no."],df["info"].str.find("Win=")+4)]
df["Len"]=[df["info"].str[y:y+2][x-1] for (x,y) in zip(df["no."],df["info"].str.find("Len=")+4)]
df["MSS"]=[df["info"].str[y:y+4][x-1] for (x,y) in zip(df["no."],df["info"].str.find("MSS=")+4)]

In [7]:
df = df.drop(['info'], axis=1)

In [72]:
df["time"]=pd.to_timedelta(df['time'].to_numpy(), 'D')

In [78]:
df.head(200)

,no.,time,source,destination,protocol,length,tcp_retransmission,SYN,ACK,Seq,Win,Len,MSS
0,1,0 days 00:00:00,172.16.0.8,64.13.134.52,TCP,58,False,True,False,0,3072,0,1460
1,2,0 days 00:02:12.969600,172.16.0.8,64.13.134.52,TCP,58,False,True,False,0,3072,0,1460
2,3,0 days 00:02:17.980800,172.16.0.8,64.13.134.52,TCP,58,False,True,False,0,2048,0,1460
3,4,0 days 00:02:22.560000,172.16.0.8,64.13.134.52,TCP,58,False,True,False,0,3072,0,1460
4,5,0 days 00:02:27.139200,172.16.0.8,64.13.134.52,TCP,58,False,True,False,0,1024,0,1460
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,2 days 00:08:36.412800,172.16.0.8,64.13.134.52,TCP,58,False,True,False,0,3072,0,1460
196,197,2 days 00:08:41.596800,172.16.0.8,64.13.134.52,TCP,58,False,True,False,0,4096,0,1460
197,198,2 days 00:08:47.040000,172.16.0.8,64.13.134.52,TCP,58,False,True,False,0,2048,0,1460
198,199,2 days 00:08:52.569600,172.16.0.8,64.13.134.52,TCP,58,False,True,False,0,4096,0,1460
